In [1]:
import pandas as pd 
import matplotlib.pyplot as plt  
import re
import numpy as np
import json

In [2]:
olddata_path = 'old data - FOW | Driving Game_October 12, 2023_23.csv'
newdata_path = 'new data - FOW | Driving Game_November 14, 2023_18.csv'
updated_path = 'updated data.csv'
old_df = pd.read_csv(olddata_path)
new_df = pd.read_csv(newdata_path)
updated_df = pd.read_csv(updated_path)

In [3]:
result_columns = [col for col in updated_df.columns if 'GameResult' in col]
# filtered_df = updated_df[result_columns]

def combine_columns(row):
    combined_data = []
    for col in result_columns:
        # Ensure the value is a string before trying to parse as JSON
        if isinstance(row[col], str) and row[col]: 
            json_data = json.loads(row[col])

            if isinstance(json_data, dict):
                combined_data.append(json_data)
            elif isinstance(json_data, list) and json_data:
                combined_data.extend(json_data)  # Append each item in the list
    return combined_data

updated_df['CombinedGameResult'] = updated_df.apply(combine_columns, axis=1)
# updated_df['CombinedGameResult'][0]

In [4]:
event_mapping = {
    0: "start game",
    1: "choose work",
    2: "enter home screen",
    3: "job available",
    4: "job expire",
    5: "start job",
    6: "guess result",
    7: "copy paste",
    8: "finish job",
    9: "rate",
    10: "choose leisure",
    11: "confirm stay in leisure",
    12: "confirm leave leisure",
    13: "didn't choose",
    14: "switch to leisure",
    15: "switch to work",
    16: "finish game",
    17: "switch location"
}

# Time based analysis
participant_time_dfs = []
for index, row in updated_df.iterrows():
    participant_df = pd.DataFrame(row['CombinedGameResult'])
    participant_df['event_name'] = participant_df['event_id'].map(event_mapping)
    participant_time_dfs.append(participant_df)

    
# Event based analysis
participant_event_dfs = []
for index, row in updated_df.iterrows():
    events = row['CombinedGameResult']
    participant_events_df = pd.DataFrame(events)
    participant_events_df['event_name'] = participant_events_df['event_id'].map(event_mapping)
    participant_events_df = participant_events_df[['event_name', 'time', 'event_id', 'count', 'core_data', 'full_message']]
    participant_event_dfs.append(participant_events_df)
    

In [5]:
participant_time_dfs[0] if participant_time_dfs else pd.DataFrame()

,time,event_id,count,core_data,full_message,event_name
0,0.000,0,1,test,test started a game with 1200 timeLimit,start game
1,2.069,10,1,None,choose leisure,choose leisure
2,25.337,11,1,None,Confirm to stay in leisure,confirm stay in leisure
3,44.073,11,2,None,Confirm to stay in leisure,confirm stay in leisure
4,63.707,11,3,None,Confirm to stay in leisure,confirm stay in leisure
...,...,...,...,...,...,...
278,1188.844,8,13,"[0, 0, 68]","Ended with: 0 mistakes, 0 earned, 68 in game t...",finish job
279,1197.400,9,13,1,Rated themselves: 1,rate
280,1197.400,2,17,"[[[0, 1071], 42, 51], [[1, 4469], 3, 28], [[2,...","Entered home screen, displayed jobs: [""i:(0,10...",enter home screen
281,1198.510,3,26,"[[1, 4469], Uber - SF]","i:(1,4469) Job Uber - SF now available",job available


In [6]:
participant_event_dfs[0] if participant_event_dfs else pd.DataFrame() 

,event_name,time,event_id,count,core_data,full_message
0,start game,0.000,0,1,test,test started a game with 1200 timeLimit
1,choose leisure,2.069,10,1,None,choose leisure
2,confirm stay in leisure,25.337,11,1,None,Confirm to stay in leisure
3,confirm stay in leisure,44.073,11,2,None,Confirm to stay in leisure
4,confirm stay in leisure,63.707,11,3,None,Confirm to stay in leisure
...,...,...,...,...,...,...
278,finish job,1188.844,8,13,"[0, 0, 68]","Ended with: 0 mistakes, 0 earned, 68 in game t..."
279,rate,1197.400,9,13,1,Rated themselves: 1
280,enter home screen,1197.400,2,17,"[[[0, 1071], 42, 51], [[1, 4469], 3, 28], [[2,...","Entered home screen, displayed jobs: [""i:(0,10..."
281,job available,1198.510,3,26,"[[1, 4469], Uber - SF]","i:(1,4469) Job Uber - SF now available"
